In [1]:
import os
import reservoirpy as rpy
from reservoirpy.datasets import mackey_glass, narma, lorenz
import matplotlib.pyplot as plt

from helpers import *

rpy.verbosity(0)
%load_ext autoreload
%autoreload 2

# Impact of Plasticity-Based Reservoir Adaptation on Spectral Radius and  Performance of ESNs

We analyze the effects of pretraining ESNs with different plasticity mechanisms. The synaptic plasticity rules that we consider are *Anti-Oja's*, *Normalized Anti-Hebbian*, *Bienenstock-Cooper-Munroe* (BCM), and *Dual-Threshold BCM* (DT-BCM). Furthermore, we apply *intrinsic plasticity* (IP).

We evaluate these pretraining methods on three benchmarks, namely a mildly chaotic Mackey-Glass, a Lorenz attractor, and a 10th-order NARMA series. 

To find out in which cases and why the methods do or do not improve the ESN's performance, we first examine the influence of the reservoir matrix' spectral radius $\rho$ on the model performance. Next, we inspect how $\rho$ is modified through the pretraining and conclude whether the change of $\rho$ might be related to the change in performance.

Some helper functions used in this notebook can be found in `helpers.py`.

The ESN that we work with is very general and not optimized to the task at hand. Its parameters are given below.

In [3]:
reservoir_params = {"units": 100, "Win": rpy.mat_gen.normal, "W": rpy.mat_gen.normal, "sr": 0.5}

## Mildly Chaotic Mackey-Glass Series

We start with the task of forecasting a mildly chaotic  ($\tau = 17$) Mackey-Glass series and generate a training and a test series of length $4000$ and $6000$, respectively. We normalize the training data to zero mean and unit variance, which proved to be beneficial in our case, and normalize the test data with the mean and variance of the training data.

In [6]:
MG_17 = mackey_glass(n_timesteps=10001)

U_train_MG, Y_train_MG, U_test_MG, Y_test_MG = normalize_data(MG_17[:4000], MG_17[1:4001], MG_17[4000:10000], MG_17[4001:10001])

We measure the performance of the ESN in terms of the $NRMSE$. Because the result depends on the random initialization of input and reservoir weights, we average the $NRMSE$ over 20 independent runs.

In [ ]:
os.makedirs("results/MG", exist_ok=True)
result_esn = avg_nrmse(n_runs=20, reservoir_params=reservoir_params,
                       U_train=U_train_MG, Y_train=Y_train_MG, U_test=U_test_MG, Y_test=Y_test_MG, warmup=500,
                       output_file="results/MG/result_esn.json", method_name="traditional ESN")

In the following, we try to improve the results of the traditional ESN by pretraining it with plasticity mechanisms. The parameters of the plasticity mechanisms are set to similar values as in the paper [Unveiling the role of plasticity rules in reservoir computing](https://www.sciencedirect.com/science/article/pii/S092523122100775X).

The results of the differently pretrained networks are saved as JSON files in the directory `results/MG`.

In [ ]:
pretrain_SP_conf_list = [{"epochs": 10, "rule": anti_oja, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": normalized_anti_hebbian, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": bcm, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": dtbcm, "params": {"eta": 1e-5, "rho": 0.1}}]
pretrain_IP_conf = {"epochs": 50, "params": {"eta": 1e-6, "mu": 0, "sigma": 1}}
pretrain_conf_list = [(pretrain_SP_conf, None) for pretrain_SP_conf in pretrain_SP_conf_list] + \
                     [(None, pretrain_IP_conf)]
output_file_list = ["results/MG/result_ao.json", "results/MG/result_nah.json", 
                    "results/MG/result_bcm.json", "results/MG/result_dtbcm.json",
                    "results/MG/result_IP.json"]
method_name_list = ["ESN pretrained with Anti-Oja's", "ESN pretrained with normalized Anti-Hebbian",
                    "ESN pretrained with BCM", "ESN pretrained with DTBCM",
                    "ESN pretrained with IP"]
evaluate_all(n_runs=20, reservoir_params=reservoir_params,
             U_train=U_train_MG, Y_train=Y_train_MG, U_test=U_test_MG, Y_test=Y_test_MG, warmup=500,
             pretrain_conf_list=pretrain_conf_list, return_sr_list=True,
             output_file_list=output_file_list, method_name_list=method_name_list)

Now we evaluate which of the pretraining methods significantly changed the results of the traditional ESN. For this evaluation, we use the Mann-Whitney $U$ test because it neither assumes normality nor equal variances. If the mean $NRMSE$ of the traditional ESN is smaller than the one of the pretrained ESN, we test whether the pretraining tends to increase the error. Otherwise, we test whether it tends to decrease it.

In [ ]:
results_dict = {}
for file_name in os.listdir("results/MG"):
    if file_name.endswith(".json"):
        with open(os.path.join("results/MG", file_name), "r") as json_file:
            results_dict[file_name[:-5]] = json.load(json_file)
            
result_esn = results_dict["result_esn"]
for file_name, result in results_dict.items():
    if result_esn["avg_nrmse"] < result["avg_nrmse"]:
        alternative = "less"
    else:
        alternative = "greater"
    if file_name != "result_esn":
        evaluate_mannwhitneyu(result_esn["nrmse_list"], result["nrmse_list"], alpha=0.05, alternative=alternative, method_names=["esn", file_name[7:]])

It turns out that pretraining with *Anti-Oja's* and *BCM* rule and with *IP* significantly improved the ESN's performance while the other pretraining methods did not significantly change the results. To find out why this is the case, we first analyze how the reservoir matrix' spectral radius influences the network's performance. The results are saved in the directory `analysis/MG`.

In [7]:
plot_nrmse_vs_sr(n_runs=20, reservoir_params=reservoir_params,
                 U_train=U_train_MG, Y_train=Y_train_MG, U_test=U_test_MG, Y_test=Y_test_MG, warmup=500,
                 output_path="analysis/MG")
# TODO: get to exec

The optimal spectral radius is $\rho = 0.75$.

Now, we analyze how the different pretraining methods affect the spectral radius. Because of the different number of epochs we create two separate subplots for the synaptic and the intrinsic plasticity rules. The resulting plot is also saved in `analysis/MG`.

In [8]:
results_dict_SP = {}
results_dict_IP = {}
for file_name in os.listdir("results/MG"):
    if file_name.endswith(".json") and file_name != "result_esn.json":
        with open(os.path.join("results/MG", file_name), "r") as json_file:
            method_name = file_name[:-5].split("_")[1]
            if method_name == "IP":
                results_dict_IP[method_name] = json.load(json_file)
            else:
                results_dict_SP[method_name] = json.load(json_file)

plot_sr_vs_epoch(results_dict_SP, results_dict_IP, sr_opt=0.75, output_file="analysis/MG/sr_vs_epoch.png")

We observe the following developments of $\rho$:

- *Anti-Oja's*: $\rho$ increases from $0.5$ to $0.81~(\pm 0.27)$.
- *Normalized Anti-Hebbian*: $\rho$ jumps from $0.5$ to $1.04$ and then remains mostly unchanged. In the last epoch, its value is $1.03~(\pm 0.05)$.
- *BCM*: $\rho$ increases slightly from $0.5$ to $0.53~(\pm 0.07)$.
- *DT-BCM*: $\rho$ remains mostly unchanged. In the last epoch, its value is $0.5~(\pm 0.002)$.
- *IP*: $\rho$ increases from $0.5$ to $0.58~(\pm 0.01)$.

## NARMA Series of Order 10

Next, we consider the task of forecasting a 10th order NARMA system. We proceed analogously to the previous task of forecasting the MG series.

In [11]:
NARMA_10 = narma(n_timesteps=10001, order=10, a1=0.3, a2=0.05, b=1.5, c=0.1)

U_train_N10 = (NARMA_10[:4000] - np.mean(NARMA_10[:4000])) / np.std(NARMA_10[:4000])
Y_train_N10 = (NARMA_10[1:4001] - np.mean(NARMA_10[1:4001])) / np.std(NARMA_10[1:4001])
U_test_N10 = (NARMA_10[4000:10000] - np.mean(NARMA_10[:4000])) / np.std(NARMA_10[:4000])
Y_test_N10 = (NARMA_10[4001:10001] - np.mean(NARMA_10[1:4001])) / np.std(NARMA_10[1:4001])

In [ ]:
os.makedirs("results/NARMA", exist_ok=True)
result_esn = avg_nrmse(n_runs=20, reservoir_params=reservoir_params,
                       U_train=U_train_N10, Y_train=Y_train_N10, U_test=U_test_N10, Y_test=Y_test_N10, warmup=500,
                       output_file="results/NARMA/result_esn.json", method_name="traditional ESN")

In [ ]:
pretrain_SP_conf_list = [{"epochs": 10, "rule": anti_oja, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": normalized_anti_hebbian, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": bcm, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": dtbcm, "params": {"eta": 1e-5, "rho": 0.1}}]
pretrain_IP_conf = {"epochs": 50, "params": {"eta": 1e-6, "mu": 0, "sigma": 1}}
pretrain_conf_list = [(pretrain_SP_conf, None) for pretrain_SP_conf in pretrain_SP_conf_list] + \
                     [(None, pretrain_IP_conf)]
output_file_list = ["results/NARMA/result_ao.json", "results/NARMA/result_nah.json", 
                  "results/NARMA/result_bcm.json", "results/NARMA/result_dtbcm.json",
                  "results/NARMA/result_IP.json"]
method_name_list = ["ESN pretrained with Anti-Oja's", "ESN pretrained with normalized Anti-Hebbian",
                    "ESN pretrained with BCM", "ESN pretrained with DTBCM",
                    "ESN pretrained with IP"]
evaluate_all(n_runs=20, reservoir_params=reservoir_params, 
             U_train=U_train_N10, Y_train=Y_train_N10, U_test=U_test_N10, Y_test=Y_test_N10, warmup=500,
             pretrain_conf_list=pretrain_conf_list, return_sr_list=True,
             output_file_list=output_file_list, method_name_list=method_name_list)

In [ ]:
results_dict = {}
for file_name in os.listdir("results/NARMA"):
    if file_name.endswith(".json"):
        with open(os.path.join("results/NARMA", file_name), "r") as json_file:
            results_dict[file_name[:-5]] = json.load(json_file)
            
result_esn = results_dict["result_esn"]
for file_name, result in results_dict.items():
    if result_esn["avg_nrmse"] < result["avg_nrmse"]:
        alternative = "less"
    else:
        alternative = "greater"
    if file_name != "result_esn":
        evaluate_mannwhitneyu(result_esn["nrmse_list"], result["nrmse_list"], alpha=0.05, alternative=alternative, method_names=["esn", file_name[7:]])

It turns out that pretraining with *Anti-Oja's*, *Normalized Anti-Hebbian* and intrinsic plasticity significantly improved the ESN's performance while the other pretraining methods did not significantly change the results.

In [12]:
plot_nrmse_vs_sr(n_runs=20, reservoir_params=reservoir_params,
                 U_train=U_train_N10, Y_train=Y_train_N10, U_test=U_test_N10, Y_test=Y_test_N10, warmup=500,
                 output_path="analysis/NARMA")

The optimal spectral radius is $\rho = 0.6$.

In [ ]:
results_dict_SP = {}
results_dict_IP = {}
for file_name in os.listdir("results/NARMA"):
    if file_name.endswith(".json") and file_name != "result_esn.json":
        with open(os.path.join("results/NARMA", file_name), "r") as json_file:
            method_name = file_name[:-5].split("_")[1]
            if method_name == "IP":
                results_dict_IP[method_name] = json.load(json_file)
            else:
                results_dict_SP[method_name] = json.load(json_file)

plot_sr_vs_epoch(results_dict_SP, results_dict_IP, sr_opt=0.6, output_file="analysis/NARMA/sr_vs_epoch.png")

We observe the following influences of the different pretraining methods on $\rho$:

- *Anti-Oja's*: $\rho$ increases from $0.5$ to $0.52~(\pm 0.05)$.
- *Normalized Anti-Hebbian*: $\rho$ jumps from $0.5$ to $1.04$ and then remains mostly unchanged. In the last epoch, its value is $1~(\pm 0.02)$.
- *BCM*: $\rho$ does not really change and always remains between $0.5$ and $0.502$. In the last epoch, its value is $0.502~(\pm 0.01)$.
- *DT-BCM*: $\rho$ does not really change. In the last epoch, its value is $0.5~(\pm 0.002)$.
- *IP*: $\rho$ increases from $0.5$ to $0.58~(\pm 0.01)$.

## Lorenz Attractor Series

Lastly, we analyze the task of forecasting a Lorenz attractor time series. In contrast to the previous two series, this data is not one-, but three-dimensional. We average the $NRMSE$ over the three dimensions and proceed analogously to above.

In [9]:
lorenz_att = lorenz(n_timesteps=10001)

U_train_LA = (lorenz_att[:4000] - np.mean(lorenz_att[:4000], axis=0)) / np.std(lorenz_att[:4000], axis=0)
Y_train_LA = (lorenz_att[1:4001] - np.mean(lorenz_att[1:4001], axis=0)) / np.std(lorenz_att[1:4001], axis=0)
U_test_LA = (lorenz_att[4000:10000] - np.mean(lorenz_att[:4000], axis=0)) / np.std(lorenz_att[0:4000], axis=0)
Y_test_LA = (lorenz_att[4001:10001] - np.mean(lorenz_att[1:4001], axis=0)) / np.std(lorenz_att[1:4001], axis=0)

In [ ]:
os.makedirs("results/lorenz", exist_ok=True)
result_esn = avg_nrmse(n_runs=20, reservoir_params=reservoir_params,
                       U_train=U_train_LA, Y_train=Y_train_LA, U_test=U_test_LA, Y_test=Y_test_LA, warmup=500,
                       output_file="results/lorenz/result_esn.json", method_name="traditional ESN")

In [ ]:
pretrain_SP_conf_list = [{"epochs": 10, "rule": anti_oja, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": normalized_anti_hebbian, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": bcm, "params": {"eta": 1e-5}},
                         {"epochs": 10, "rule": dtbcm, "params": {"eta": 1e-5, "rho": 0.1}}]
pretrain_IP_conf = {"epochs": 50, "params": {"eta": 1e-6, "mu": 0, "sigma": 1}}
pretrain_conf_list = [(pretrain_SP_conf, None) for pretrain_SP_conf in pretrain_SP_conf_list] + \
                     [(None, pretrain_IP_conf)]
output_file_list = ["results/lorenz/result_ao.json", "results/lorenz/result_nah.json", 
                  "results/lorenz/result_bcm.json", "results/lorenz/result_dtbcm.json",
                  "results/lorenz/result_IP.json"]
method_name_list = ["ESN pretrained with Anti-Oja's", "ESN pretrained with normalized Anti-Hebbian",
                    "ESN pretrained with BCM", "ESN pretrained with DTBCM",
                    "ESN pretrained with IP"]
evaluate_all(n_runs=20, reservoir_params=reservoir_params, 
             U_train=U_train_LA, Y_train=Y_train_LA, U_test=U_test_LA, Y_test=Y_test_LA, warmup=500,
             pretrain_conf_list=pretrain_conf_list, return_sr_list=True,
             output_file_list=output_file_list, method_name_list=method_name_list)

In [ ]:
results_dict = {}
for file_name in os.listdir("results/lorenz"):
    if file_name.endswith(".json"):
        with open(os.path.join("results/lorenz", file_name), "r") as json_file:
            results_dict[file_name[:-5]] = json.load(json_file)
            
result_esn = results_dict["result_esn"]
for file_name, result in results_dict.items():
    if result_esn["avg_nrmse"] < result["avg_nrmse"]:
        alternative = "less"
    else:
        alternative = "greater"
    if file_name != "result_esn":
        evaluate_mannwhitneyu(result_esn["nrmse_list"], result["nrmse_list"], alpha=0.05, alternative=alternative, method_names=["esn", file_name[7:]])

It turns out that pretraining with *Anti-Oja's* and *Normalized Anti-Hebbian* rule significantly worsened the ESN's performance while pretraining with *BCM* and intrinsic plasticity significantly improved it.

In [13]:
plot_nrmse_vs_sr(n_runs=20, reservoir_params=reservoir_params,
                 U_train=U_train_LA, Y_train=Y_train_LA, U_test=U_test_LA, Y_test=Y_test_LA, warmup=500,
                 output_path="analysis/lorenz")

The optimal spectral radius is $\rho=0.4$.

In [ ]:
results_dict_SP = {}
results_dict_IP = {}
for file_name in os.listdir("results/lorenz"):
    if file_name.endswith(".json") and file_name != "result_esn.json":
        with open(os.path.join("results/lorenz", file_name), "r") as json_file:
            method_name = file_name[:-5].split("_")[1]
            if method_name == "IP":
                results_dict_IP[method_name] = json.load(json_file)
            else:
                results_dict_SP[method_name] = json.load(json_file)

plot_sr_vs_epoch(results_dict_SP, results_dict_IP, sr_opt=0.4, output_file="analysis/lorenz/sr_vs_epoch.png")

We observe the following influences of the different pretraining methods on $\rho$:

- *Anti-Oja's*: $\rho$ increases from $0.5$ to $1.33~(\pm 0.19)$.
- *Normalized Anti-Hebbian*: $\rho$ jumps from $0.5$ to $1.06$ where it remains mostly unchanged over a few epochs until it continues to increase to $1.13~(\pm 0.13)$.
- *BCM*: $\rho$ increases from $0.5$ to $0.6~(\pm 0.11)$.
- *DT-BCM*: $\rho$ does not really change. In the last epoch, its value is $0.5~(\pm 0.01)$.
- *IP*: $\rho$ increases from $0.5$ to $0.56~(\pm 0.01)$.